In [1]:
import os
import pandas as pd
import alpaca_trade_api as tradeapi
import requests
import json

from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from newsapi import NewsApiClient

In [2]:
# Set APIs keys
load_dotenv()
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
news_api_key = os.getenv("NEWS_API_KEY")

In [3]:
#Getting ticker to evaluate 
tickers = input("Enter the stock to evaluate: ") 

def get_symbol(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)

    result = requests.get(url).json()

    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

company = get_symbol(f"{tickers}")

print(f"You entered {tickers}: {company}. Let's evaluate it...") 

Enter the stock to evaluate:  GOOG


You entered GOOG: Alphabet Inc.. Let's evaluate it...


In [4]:
newsapi = NewsApiClient(api_key=f"{news_api_key}")
all_articles = newsapi.get_everything(q=f"{company}",
                                      from_param='2020-09-31',
                                      to='2020-10-31',
                                      language='en',
                                      sort_by='relevancy')
print(json.dumps(all_articles, indent=4))

{
    "status": "ok",
    "totalResults": 1543,
    "articles": [
        {
            "source": {
                "id": "reuters",
                "name": "Reuters"
            },
            "author": "Reuters Staff",
            "title": "Alphabet Inc's Google launches Google TV, new Chromecast - Reuters India",
            "description": "Alphabet Inc's Google on Wednesday launched Google TV, which would show content from streaming services including Netflix Inc and Walt Disney Co's Disney+.",
            "url": "https://in.reuters.com/article/alphabet-products-idINKBN26M48O",
            "urlToImage": "https://static.reuters.com/resources/r/?m=02&d=20201001&t=2&i=1535630528&r=LYNXMPEG902XE&w=800",
            "publishedAt": "2020-10-01T02:34:00Z",
            "content": "By Reuters Staff\r\nFILE PHOTO: After the company announced it would extend its coronavirus work-from-home order until summer 2021, a Google sign is shown at one of the company's office complexes in Ir\u2026 [+10

In [5]:
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [6]:
timeframe ='1D'

In [7]:
start_date = pd.Timestamp("2010-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-10-29", tz="America/New_York").isoformat()

In [8]:
df_ticker = api.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

In [9]:
df_ticker.head()

GOOG                                 
                             open    high     low   close   volume
2010-01-04 00:00:00-05:00  313.47  314.75  312.12  313.40  1622642
2010-01-05 00:00:00-05:00  313.59  313.92  310.77  312.01  2872417
2010-01-06 00:00:00-05:00  312.93  312.93  303.18  304.13  3844995
2010-01-07 00:00:00-05:00  304.70  305.00  296.32  297.00  6279974
2010-01-08 00:00:00-05:00  296.00  301.62  294.55  300.56  4564872

In [10]:
MC_even_dist = MCSimulation(
    portfolio_data = df_ticker,
    weights = [1],
    num_simulation = 1000,
    num_trading_days = 252*5
)

In [11]:
MC_even_dist.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.002596,1.008319,0.998269,0.970070,0.994048,1.008821,1.002135,0.976873,1.000656,0.970611,...,0.962343,1.003591,0.998421,0.993120,0.993705,0.994254,0.999255,1.015186,0.962117,1.003327
2,1.011783,0.997138,0.960751,0.969028,0.998099,0.999154,1.008313,0.968166,1.006073,0.997695,...,0.987588,1.008536,1.019749,0.990158,0.997031,0.994825,1.030852,1.027744,0.980277,0.977746
3,1.005990,1.019539,0.968903,0.979977,0.979595,0.982374,1.022339,0.989825,0.993671,0.984948,...,0.988376,0.981921,1.023940,1.002905,1.021368,0.995171,1.019981,1.053277,0.984777,0.979232
4,1.013056,1.014586,0.993777,0.975042,0.971916,0.955513,0.997428,0.993857,0.991789,0.986317,...,0.965420,0.985504,0.993225,0.991681,1.022342,0.995974,1.013066,1.058182,0.987590,1.001565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,2.378243,2.280866,1.312551,0.991352,3.249125,1.752548,4.799715,1.278597,1.790815,2.086493,...,3.017957,2.656053,1.376531,1.361824,1.380048,1.946938,3.456443,4.411436,1.491010,4.707659
1257,2.420116,2.255866,1.294398,0.970734,3.274280,1.784495,4.784824,1.273596,1.774387,2.005963,...,2.915203,2.661128,1.394091,1.399767,1.402658,1.968746,3.439202,4.613970,1.540295,4.589420
1258,2.463977,2.277551,1.293252,0.954853,3.158265,1.790293,4.751696,1.266463,1.762288,2.007019,...,2.838400,2.645154,1.335867,1.431917,1.400818,2.003894,3.423460,4.660447,1.526561,4.638115
1259,2.459647,2.235767,1.302931,0.946595,3.079954,1.819372,4.842136,1.273168,1.762711,2.059179,...,2.845449,2.626189,1.349752,1.431086,1.436294,1.953120,3.433005,4.611665,1.504519,4.614976


In [12]:
even_tbl = MC_even_dist.summarize_cumulative_return()
print(even_tbl)

count           1000.000000
mean               2.511721
std                1.527044
min                0.368143
25%                1.453489
50%                2.118129
75%                3.163337
max               10.434037
95% CI Lower       0.617995
95% CI Upper       6.567283
Name: 1260, dtype: float64
